In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit
#import mplhep as hep
%load_ext autoreload
%autoreload 2

In [ ]:
from unc_stdy.utils import select_sectors
from unc_stdy.fit_func import standard_error, std_err_sys
from unc_stdy.common_objects import wheel_stations_df

In [ ]:
#load datagrame produced in dt_uncertainty
result_df = pd.read_csv("Result.csv")

In [ ]:
def calc_sys_wheel_station(row, coordinate, ax1=0, ax2=0, min_nMuons=100):
    '''Function to calculate systematics for one row of the dataframe.'''
    #make temporary df out of wheel and station seleciton
    print("Solving for {}/{}".format(row.wheel, row.station))
    tdf = select_sectors(result_df, row.wheel, row.station)
    tdf = tdf[tdf.nMuons_mean>min_nMuons]
    #fit stat curve
    hesse_unc_string = "hesse_unc_{}".format(coordinate)
    popt_stat, pcov_stat = curve_fit(standard_error, tdf.nMuons_mean, tdf[hesse_unc_string])
    #fit sys curve
    std_name = "{} std".format(coordinate)
    def fit_sys(x, sys, scale): 
        '''Wrapper to account for seperate fit. Scale added in '''
        return scale * std_err_sys(x, *popt_stat, sys)
    popt_sys, pcov_sys = curve_fit(fit_sys, tdf.nMuons_mean, tdf[std_name], bounds=(0,999))
    #create x range for plotting
    x = np.linspace(min(tdf.nMuons_mean), max(tdf.nMuons_mean), 100)   
    #plot fit stats
    if ax1:
        ax1.scatter(tdf.nMuons_mean, tdf[hesse_unc_string], label="Minuit Unc.")
        ax1.plot(x, standard_error(x,*popt_stat), label="Minuit Unc. Fit")
    #plot fit sys
    if ax2:
        ax2.scatter(tdf.nMuons_mean, tdf[std_name], label="Measured Unc.")
        ax2.plot(x, fit_sys(x,*popt_sys), label="Measured Unc. Fit")    
    #
    #format ax
    #
    if ax1: ax1.legend()
    if ax2: ax2.legend()
    return pd.Series({"wheel": row.wheel, "station":row.station, "sigma": popt_stat[0], "sys": popt_sys[0],  "scale": popt_sys[1]})

In [ ]:
# figure to show all fits for all wheel/station
fit_list = []
fig, ax = plt.subplots(4,3, figsize=(15,15))
for i, station_axs in enumerate(ax):
    #reverse station order for aesthetics
    station = 4 - i
    for j, ax in enumerate(station_axs):
        wheel = j
        tdf = pd.Series({"wheel": wheel, "station":station})
        #run fit
        fit_series = calc_sys_wheel_station(tdf, "x", ax1=ax, ax2=ax)
        fit_list.append(fit_series)
        #format ax
        ax.set_xlabel("n Muons (average)")
        ax.set_ylabel("[cm])")
        ax.legend(title="{}/{}".format(wheel, station))
fig.savefig("output/wheel_station_fits_method2.pdf")

In [ ]:
# So elegant, but it's hard to draw the fit plots this way:
#fits = wheel_stations_df.apply(lambda x: calc_sys_wheel_station(x, "x"), axis=1)
fits = pd.DataFrame(fit_list)
#scale the sys param by the scale factor
fits['scalled_sys'] = fits.sys*fits.scale

In [ ]:
#plot sys and stat plots as function of wheel and station
wheel_bins = np.linspace(-.5,2.5, 4)
station_bins = np.linspace(.5,4.5, 5)
fig, ax = plt.subplots(1,2, figsize=(12,6))
#sys plot
counts, xedges, yedges, im = ax[0].hist2d(fits.wheel, fits.station, weights=fits.scalled_sys,
          bins = [wheel_bins, station_bins])
fig.colorbar(im, ax=ax[0])
#stat plot
counts, xedges, yedges, im = ax[1].hist2d(fits.wheel, fits.station, weights=fits.sigma,
          bins = [wheel_bins, station_bins])
fig.colorbar(im, ax=ax[1])
#format plots
def format_ax(title, ax):
    ax.set_title(title) 
    ax.set_yticks([1,2,3,4])
    ax.set_ylabel('Station')
    ax.set_xticks([0,1,2])
    ax.set_xlabel('Wheel')
format_ax("Sys. Width [cm]", ax[0])
format_ax("Sigma [cm] (statistical uncertainty)", ax[1])
fig.savefig('output/sys_sigma_overview_method2.pdf')

In [ ]:
fits